### Notebook responsável por gerenciar as operações com o banco de dados

In [9]:
import datetime
from sqlalchemy import create_engine

In [10]:
user='postgres'
passwd='postgres'
host='postgresql'
port='5432'
db='cartola'
url = 'postgresql://{user}:{passwd}@{host}:{port}/{db}'.format(user=user, passwd=passwd, host=host, port=port, db=db)

In [11]:
def create_database_session():
    engine = create_engine(url)
    return engine

In [13]:
def create_table_match_if_not_exists(engine):
    engine.execute('''
        CREATE TABLE IF NOT EXISTS match (
            match_id integer, 
            turn smallint,
            home_id smallint,
            visitor_id smallint,
            date varchar(20),
            timestamp bigint,
            local varchar(80),
            valid boolean,
            home_goal smallint,
            visitor_goal smallint,
            year smallint
        )'''
    )

In [ ]:
def create_table_club_if_not_exists(engine):
    engine.execute('''
        CREATE TABLE IF NOT EXISTS club (
            id smallint, 
            name varchar(80),
            initials varchar(3),
            year smallint
        )'''
    )

In [24]:
def alter_table_match_if_column_year_not_exists(engine):
    engine.execute('ALTER TABLE match ADD COLUMN IF NOT EXISTS year INTEGER NOT NULL DEFAULT 2021')

In [ ]:
def alter_table_club_if_column_year_not_exists(engine):
    engine.execute('ALTER TABLE club ADD COLUMN IF NOT EXISTS year INTEGER NOT NULL DEFAULT 2021')

In [1]:
def erase_data(engine, table):
    year = datetime.date.today().year
    engine.execute('delete from {} where (year IS NULL OR year = {})'.format(table, year))

In [33]:
def save_data(engine, table_name, dataframe):
    dataframe.to_sql(table_name, con=engine, index=False, if_exists='replace')

In [16]:
def count_data(engine, table):
    return engine.execute('SELECT count(*) FROM {}'.format(table))

In [ ]:
def get_matches_dataframe(engine):
    query = 'select * from match;'
    return __get_dataframe__(engine, query)

In [ ]:
def get_scouts_dataframe(engine):
    query = 'select * from scouts;'
    return __get_dataframe__(engine, query)

In [ ]:
def get_clubs_dataframe(engine):
    query = 'select * from club;'
    return __get_dataframe__(engine, query)

In [3]:
def get_mercado_dataframe(engine):
    query = 'select * from mercado;'
    return __get_dataframe__(engine, query)

In [ ]:
def __get_dataframe__(engine, query):
    return pd.read_sql(query, con=engine)

In [2]:
def prepare_database():
    engine = create_database_session()
    create_table_match_if_not_exists(engine)
    alter_table_match_if_column_year_not_exists(engine)
    alter_table_club_if_column_year_not_exists(engine)
    try:
      erase_data(engine, 'match')
      erase_data(engine, 'mercado')
    except:
        print("Tabela ainda nao existe")
    return engine